# Data exploration 

---

Group name: O

---


## Introduction

*This section includes a short description of the data* 

## Setup

In [7]:
import pandas as pd
import altair as alt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

alt.data_transformers.disable_max_rows()


ModuleNotFoundError: No module named 'altair'

## Data

## Import data

In [8]:
DATAGRADSTUDENTS = "https://raw.githubusercontent.com/FabioRaab/bigData-HW1/main/data/external/grad-students.csv"

### Data structure

In [9]:
df = pd.read_csv(DATAGRADSTUDENTS)
print(df)

df.info()

     Major_code                                       Major  \
0          5601                       CONSTRUCTION SERVICES   
1          6004           COMMERCIAL ART AND GRAPHIC DESIGN   
2          6211                      HOSPITALITY MANAGEMENT   
3          2201      COSMETOLOGY SERVICES AND CULINARY ARTS   
4          2001                  COMMUNICATION TECHNOLOGIES   
..          ...                                         ...   
168        5203                       COUNSELING PSYCHOLOGY   
169        5202                         CLINICAL PSYCHOLOGY   
170        6106     HEALTH AND MEDICAL PREPARATORY PROGRAMS   
171        2303                   SCHOOL STUDENT COUNSELING   
172        2301  EDUCATIONAL ADMINISTRATION AND SUPERVISION   

                          Major_category  Grad_total  Grad_sample_size  \
0    Industrial Arts & Consumer Services        9173               200   
1                                   Arts       53864               882   
2                    

### Data corrections

In [10]:
df['Grad_median'] = df['Grad_median'].astype("category")
df['Nongrad_median'] = df['Nongrad_median'].astype("category")

In [ ]:
df.info()

### Variable lists

In [ ]:
# make a list of variables you want to use
var_list = ['Grad_median', 'Nongrad_median']

In [ ]:
source = df[var_list]

In [ ]:
source

In [ ]:
alt.Chart(source).mark_bar().encode(
    alt.X("Grad_median", bin=True),
    y='count()',
)

In [ ]:
# contingency table for application type and homeownership.
pd.crosstab(df['Grad_median'],    # rows: Grad_median
            df['Nongrad_median'],    # columns: Nongrad_median
            margins=True)          # with total count

In [ ]:
chart = alt.Chart(source).mark_bar().encode(
    x=alt.X('Grad_median',
        sort='-y' ),
    y=alt.Y('count(Nongrad_median)')
)
chart

In [ ]:
#chart

#chart.properties( 
 #   title= 'Predicted Score in first halftime',
  #  width= 300,
   # height= 200)